In [1]:
from lighteval.tasks import lighteval_task
from datasets import load_dataset


def replace_load_dataset(path, name, revision):
    if path == "HuggingFaceH4/MATH-500":
        return load_dataset("/mnt/sata2tb/leo/NLP/GRDPO/datasets/math500")
    else:
        return load_dataset(path=path, name=name, revision=revision)
    
lighteval_task.load_dataset = replace_load_dataset

In [2]:
from transformers import AutoModelForCausalLM
from lighteval.logging.evaluation_tracker import EvaluationTracker
from lighteval.models.transformers.transformers_model import TransformersModel, TransformersModelConfig
from lighteval.pipeline import ParallelismManager, Pipeline, PipelineParameters

test_name = "basemodel"

MODEL_NAME = "/home/leo/NLP/models/Qwen-VL/Qwen2.5-1.5B-Instruct"
BENCHMARKS = "math_500"

evaluation_tracker = EvaluationTracker(output_dir="./results_" + test_name)

pipeline_params = PipelineParameters(
    launcher_type=ParallelismManager.NONE,
    # max_samples=3
)

model = AutoModelForCausalLM.from_pretrained(
  MODEL_NAME, device_map="auto"
).eval()
config = TransformersModelConfig(model_name=MODEL_NAME, batch_size=1)
model = TransformersModel.from_model(model, config)

pipeline = Pipeline(
    model=model,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    tasks=BENCHMARKS,
)

results = pipeline.evaluate()
pipeline.show_results()
results = pipeline.get_results()

Careful, the task math_500 is using evaluation data to build the few shot examples.
You cannot select the number of dataset splits for a generative evaluation at the moment. Automatically inferring.
Splits:   0%|          | 0/1 [28:26<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 120.00 MiB. GPU 0 has a total capacity of 23.51 GiB of which 55.00 MiB is free. Including non-PyTorch memory, this process has 21.20 GiB memory in use. Of the allocated memory 18.44 GiB is allocated by PyTorch, and 2.31 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)